In [ ]:
!pip install googletrans==3.1.0a0 

In [ ]:
from googletrans import Translator
import pandas as pd
from tqdm import tqdm
from ast import literal_eval

translator = Translator()

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
def str_to_dict(dataset):
    def func(obj):
        List = literal_eval(obj)
        return List
    
    out = dataset.copy()
    out['subject_entity'] = dataset['subject_entity'].apply(func)
    out['object_entity'] = dataset['object_entity'].apply(func)
    
    return out
    
out_dataset = str_to_dict(df)
out_dataset

In [ ]:
new_df=pd.DataFrame()
ids=[]
sentence=[]
se=[]
oe=[]
lab=[]
source=[]

In [ ]:
for i in tqdm(list(out_dataset['id']), desc='Back Translation', total=len(out_dataset)):
      subject_ent = out_dataset['subject_entity'][i]
      object_ent = out_dataset['object_entity'][i]
      sen = out_dataset['sentence'][i]
      
      new = translator.translate(translator.translate(sen, src='ko', dest='en').text, src='en', dest='ko').text

      if new.find(subject_ent['word'])>=0 and new.find(object_ent['word'])>=0:
        subject_ent['start_idx'] = new.index(subject_ent['word'])
        subject_ent['end_idx'] = subject_ent['start_idx'] + len(subject_ent['word']) - 1
        object_ent['start_idx'] = new.index(object_ent['word'])
        object_ent['end_idx'] = object_ent['start_idx'] + len(object_ent['word']) - 1

        subject_ent=str(subject_ent)
        object_ent=str(object_ent)

        ids.append(i)
        sentence.append(new)
        se.append(subject_ent)
        oe.append(object_ent)
        lab.append(out_dataset['label'][i])
        source.append(out_dataset['source'][i])

new_df=pd.DataFrame(ids)
new_df.columns=['id']
new_df['sentence'],new_df['subject_entity'],new_df['object_entity'],new_df['label'],new_df['source']=sentence,se,oe,lab,source
new_df.to_csv("translated_train.csv",index=False)
new_df

In [ ]:
df = pd.read_csv('train.csv')
df2 = pd.read_csv('translated_train.csv')

In [ ]:
new_df=pd.concat([df,df2])
new_df

In [ ]:
new_df.to_csv("translated+train.csv",index=False)